In [2]:
import scanpy as sc
import pandas as pd

In [3]:
orig_adata = sc.read_h5ad("/home/igarzonalva/Proyecto_SC_TNBC/GSE161529/LabelTransfer/adatas/MyeloidAdata.h5ad")

In [4]:
orig_adata = orig_adata.raw.to_adata()

In [5]:
orig_adata

AnnData object with n_obs × n_vars = 352734 × 45453
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'harm_study', 'harm_healthy.tissue', 'harm_tumor.site', 'harm_sample.type', 'harm_condition', 'harm_tumor.type', 'harm_cd45pos', 'harm_healthy.pat', 'percent.mt', 'ratio_nCount_nFeature', 'batch', 'X_scvi_batch', 'X_scvi_labels', 'X_scvi_local_l_mean', 'X_scvi_local_l_var', 'leiden_0.2', 'leiden_0.4', 'leiden_0.6', 'leiden_0.8', 'leiden_1', 'leiden_1.2', 'leiden_1.4', 'author_first_cell_type', 'author_cell_type', 'cnv_score', 'organism_ontology_term_id', 'donor_id', 'development_stage_ontology_term_id', 'sex_ontology_term_id', 'self_reported_ethnicity_ontology_term_id', 'disease_ontology_term_id', 'tissue_ontology_term_id', 'cell_type_ontology_term_id', 'suspension_type', 'assay_ontology_term_id', 'cell_id', 'dup', 'is_primary_data', 'tissue_type', 'cell_type', 'assay', 'disease', 'organism', 'sex', 'tissue', 'self_reported_ethnicity', 'development_stage', 'observation_joinid'
    v

In [6]:
ensembl_version_dict = {'105': 'http://www.ensembl.org',

                        '104': 'http://may2021.archive.ensembl.org/',

                        '103': 'http://feb2021.archive.ensembl.org/',

                        '102': 'http://nov2020.archive.ensembl.org/',

                        '101': 'http://aug2020.archive.ensembl.org/',

                        '100': 'http://apr2020.archive.ensembl.org/',

                        '99': 'http://jan2020.archive.ensembl.org/',

                        '98': 'http://sep2019.archive.ensembl.org/',

                        '97': 'http://jul2019.archive.ensembl.org/',

                        '96': 'http://apr2019.archive.ensembl.org/',

                        '95': 'http://jan2019.archive.ensembl.org/',

                        '94': 'http://oct2018.archive.ensembl.org/',

                        '93': 'http://jul2018.archive.ensembl.org/',

                        '92': 'http://apr2018.archive.ensembl.org/',

                        '91': 'http://dec2017.archive.ensembl.org/',

                        '90': 'http://aug2017.archive.ensembl.org/',

                        '89': 'http://may2017.archive.ensembl.org/',

                        '88': 'http://mar2017.archive.ensembl.org/',

                        '87': 'http://dec2016.archive.ensembl.org/',

                        '86': 'http://oct2016.archive.ensembl.org/',

                        '80': 'http://may2015.archive.ensembl.org/',

                        '77': 'http://oct2014.archive.ensembl.org/',

                        '75': 'http://feb2014.archive.ensembl.org/',

                        '54': 'http://may2009.archive.ensembl.org/'}



In [7]:
import pybiomart as pbm

In [8]:
def test_ensembl_host(adata, host):

    dataset = pbm.Dataset(name='hsapiens_gene_ensembl', host=host)
    
    qp = dataset.query(attributes=['ensembl_gene_id', 'external_gene_name'])
    print(f'Nº of genes in db: {len(qp)}')
    
    df_var = pd.DataFrame(adata.var_names.values, columns=["Gene stable ID"])
    print(f'Nº of genes in dataset: {len(df_var)}')
    
    df_mapped = df_var.merge(qp, how="inner", on="Gene stable ID")
    print(f'Nº of mapped genes to db: {len(df_mapped)}')
    
    df_valid = df_mapped[~df_mapped["Gene name"].isna()]
    ov = len(df_valid)
    print(f'Nº of valid mapped genes to db: {ov}')
    
    return(ov)


In [9]:
n_overlap = {}

for version in ensembl_version_dict.keys():

    print(f'host: {version}')

    try:

        n_overlap[version] =  test_ensembl_host(orig_adata, ensembl_version_dict[version])

    except:

        print('Host not reachable')

v = sorted(n_overlap.items(), key=lambda item: item[1], reverse=True)[0][0]

host_to_use = ensembl_version_dict[v]
print(f"version: {v} has the largest overlap, use {host_to_use} as biomart host")



host: 105
Nº of genes in db: 86402
Nº of genes in dataset: 45453
Nº of mapped genes to db: 44653
Nº of valid mapped genes to db: 35967
host: 104
Nº of genes in db: 67128
Nº of genes in dataset: 45453
Nº of mapped genes to db: 45453
Nº of valid mapped genes to db: 35225
host: 103
Nº of genes in db: 67130
Nº of genes in dataset: 45453
Nº of mapped genes to db: 45453
Nº of valid mapped genes to db: 45453
host: 102
Nº of genes in db: 67139
Nº of genes in dataset: 45453
Nº of mapped genes to db: 45451
Nº of valid mapped genes to db: 45451
host: 101
Nº of genes in db: 67130
Nº of genes in dataset: 45453
Nº of mapped genes to db: 45451
Nº of valid mapped genes to db: 45451
host: 100
Nº of genes in db: 67149
Nº of genes in dataset: 45453
Nº of mapped genes to db: 45451
Nº of valid mapped genes to db: 45451
host: 99
Nº of genes in db: 67140
Nº of genes in dataset: 45453
Nº of mapped genes to db: 45448
Nº of valid mapped genes to db: 45448
host: 98
Nº of genes in db: 67087
Nº of genes in dataset

In [10]:
def map_genes(adata, host):

    dataset = pbm.Dataset(name='hsapiens_gene_ensembl', host=host)
    
    qp = dataset.query(attributes=['ensembl_gene_id', 'external_gene_name'])
    print(f'Nº of genes in db: {len(qp)}')
    
    df_var = pd.DataFrame(adata.var_names.values, columns=["Gene stable ID"])
    print(f'Nº of genes in dataset: {len(df_var)}')
    
    df_mapped = df_var.merge(qp, how="inner", on="Gene stable ID")
    print(f'Nº of mapped genes to db: {len(df_mapped)}')
    
    df_valid = df_mapped[~df_mapped["Gene name"].isna()]

    
    gene_order = df_valid["Gene stable ID"].tolist()

    adata = adata[:, gene_order].copy() 

    adata.var_names = df_valid['Gene name'].values

    return(adata)   
    

In [11]:
mod_adata = map_genes(orig_adata, host_to_use)

Nº of genes in db: 67130
Nº of genes in dataset: 45453
Nº of mapped genes to db: 45453


In [12]:
mod_adata.var_names

Index(['TSPAN6', 'TNMD', 'DPM1', 'SCYL3', 'C1orf112', 'FGR', 'CFH', 'FUCA2',
       'GCLC', 'NFYA',
       ...
       'AP002991.2', 'AL355338.1', 'AC005618.4', 'AC126768.3', 'C3orf36',
       'C8orf44', 'C8orf44-SGK3', 'SNORA74C-2', 'XGY2', 'FLJ43315'],
      dtype='object', length=45453)

In [13]:
query_adata = sc.read_h5ad("/home/igarzonalva/Proyecto_SC_TNBC/GSE161529/Integration/adata/adata_scanvi_predictions.h5ad")

In [14]:
len(set(query_adata.var_names).intersection(set(mod_adata.var_names)))

17262

In [15]:
common_genes = [gene for gene in mod_adata.var_names if gene in query_adata.var_names]

In [16]:
common_genes

['TSPAN6',
 'DPM1',
 'SCYL3',
 'C1orf112',
 'FGR',
 'CFH',
 'FUCA2',
 'GCLC',
 'NFYA',
 'STPG1',
 'NIPAL3',
 'LAS1L',
 'ENPP4',
 'SEMA3F',
 'CFTR',
 'ANKIB1',
 'CYP51A1',
 'KRIT1',
 'RAD52',
 'BAD',
 'LAP3',
 'CD99',
 'HS3ST1',
 'AOC1',
 'MAD1L1',
 'LASP1',
 'SNX11',
 'TMEM176A',
 'M6PR',
 'KLHL13',
 'CYP26B1',
 'ICA1',
 'DBNDD1',
 'ALS2',
 'CASP10',
 'CFLAR',
 'TFPI',
 'NDUFAF7',
 'RBM5',
 'MTMR7',
 'SLC7A2',
 'ARF5',
 'SARM1',
 'POLDIP2',
 'PLXND1',
 'AK2',
 'CD38',
 'FKBP4',
 'KDM1A',
 'RBM6',
 'CAMKK1',
 'RECQL',
 'VPS50',
 'HSPB6',
 'ARHGAP33',
 'NDUFAB1',
 'PDK4',
 'ZMYND10',
 'ARX',
 'SLC25A13',
 'ST7',
 'CDC27',
 'CALCR',
 'HCCS',
 'DVL2',
 'PRSS22',
 'UPF1',
 'SKAP2',
 'SLC25A5',
 'MCUB',
 'HOXA11',
 'POLR2J',
 'DHX33',
 'MEOX1',
 'THSD7A',
 'LIG3',
 'RPAP3',
 'ACSM3',
 'REXO5',
 'CIAPIN1',
 'SPPL2B',
 'FAM214B',
 'COPZ2',
 'PRKAR2B',
 'MSL3',
 'CREBBP',
 'TSPOAP1',
 'MPO',
 'GCFC2',
 'WDR54',
 'CROT',
 'KMT2E',
 'RHBDD2',
 'SOX8',
 'IBTK',
 'ZNF195',
 'MYCBP2',
 'FBXL3',
 'IT

In [21]:
query_adata.var_names

Index(['A1BG', 'A1BG-AS1', 'A2M', 'A2M-AS1', 'A2ML1', 'A4GALT', 'AAAS', 'AACS',
       'AADAC', 'AADAT',
       ...
       'ZW10', 'ZWILCH', 'ZWINT', 'ZXDA', 'ZXDB', 'ZXDC', 'ZYG11A', 'ZYG11B',
       'ZYX', 'ZZEF1'],
      dtype='object', length=18088)

In [17]:
query_matched = query_adata[:,common_genes].copy()

/home/igarzonalva/.conda/envs/scvi-env/lib/python3.12/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [22]:
query_matched.var_names

Index(['TSPAN6', 'DPM1', 'SCYL3', 'C1orf112', 'FGR', 'CFH', 'FUCA2', 'GCLC',
       'NFYA', 'STPG1',
       ...
       'AC068282.1', 'AC092803.2', 'AL591845.1', 'AL161935.3', 'AL512625.3',
       'AC239799.2', 'AC008403.3', 'AC061961.1', 'AL355338.1', 'C8orf44'],
      dtype='object', length=17269)

In [27]:
mod_adata.var_names_make_unique()

In [23]:
mod_adata.var_names_make_unique()

In [24]:
mod_adata_matched = mod_adata[:,common_genes].copy()

/home/igarzonalva/.conda/envs/scvi-env/lib/python3.12/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [26]:
mod_adata_matched.var_names_make_unique()

In [15]:
len(query_adata.var_names)

18088

In [16]:
len(query_matched.var_names)

17269

In [28]:
len(set(query_matched.var_names).intersection(set(mod_adata_matched.var_names)))

17262

In [21]:
query_matched.obs.GennAnno_ScAnvi.cat.categories

Index(['B cells', 'Cycling cells', 'Endothelial', 'Epithelial', 'Mast cells',
       'Mesenchymal', 'Myeloid', 'Plasma cells', 'T cells'],
      dtype='object')

In [22]:
myeloid_query = query_matched[query_matched.obs.GennAnno_ScAnvi.isin(['Myeloid','Mast cells']),:]

In [23]:
myeloid_query

View of AnnData object with n_obs × n_vars = 14380 × 17269
    obs: 'sample', 'n_genes_by_counts', 'total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_mt', 'pct_counts_mt', 'complexity', 'doublet_scores', 'predicted_doublets', 'n_genes', 'n_counts', 'celltypist_labels_Immune_All_High', 'celltypist_scores_Immune_All_High', 'celltypist_labels_Cells_Adult_Breast', 'celltypist_scores_Cells_Adult_Breast', 'batch', 'subtype', 'leiden', 'celltypist_labels_Immune_All_High_highconf', 'GenAnno', 'GenAnnoV1', 'GenAnnoV2', 'celltypist_mapped', 'GenAnnoHighConf', 'cell_type', 'predicted_labels', 'GennAnno_ScAnvi'
    var: 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm', 'highly_variable_nbatches'
    uns: 'GenAnnoHighConf_colors', 'GenAnnoV1_colors', 'GenAnnoV2_colors', 'GennAnno_ScAnvi_colors', 'celltypist_labels_Immune_All_High_highconf_colors', 'hvg', 'leiden', '

In [1]:
myeloid_query

NameError: name 'myeloid_query' is not defined